In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
import tensorflow as tf
from tensorflow import keras as k
from keras import layers as l
import xgboost as xgb


In [31]:
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')

In [32]:
df.drop(columns=['Id'], axis = 1, inplace = True)
df.describe()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,...,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,...,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,...,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,...,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,...,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,...,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,...,6.238814,1.512060,0.535067,25.608406,127.591671,19035.709240,36.863947,67.931664,21.978000,0.000000
max,6.161666,28688.187660,1910.123198,630.518230,178.943634,38.270880,10.315851,38.971568,1463.693448,53060.599240,...,137.932739,1244.227020,31.365763,135.781294,1497.351958,143790.071200,81.210825,191.194764,21.978000,1.000000


In [33]:
df.dtypes

AB       float64
AF       float64
AH       float64
AM       float64
AR       float64
AX       float64
AY       float64
AZ       float64
BC       float64
BD       float64
BN       float64
BP       float64
BQ       float64
BR       float64
BZ       float64
CB       float64
CC       float64
CD       float64
CF       float64
CH       float64
CL       float64
CR       float64
CS       float64
CU       float64
CW       float64
DA       float64
DE       float64
DF       float64
DH       float64
DI       float64
DL       float64
DN       float64
DU       float64
DV       float64
DY       float64
EB       float64
EE       float64
EG       float64
EH       float64
EJ        object
EL       float64
EP       float64
EU       float64
FC       float64
FD       float64
FE       float64
FI       float64
FL       float64
FR       float64
FS       float64
GB       float64
GE       float64
GF       float64
GH       float64
GI       float64
GL       float64
Class      int64
dtype: object

In [34]:
cols = list(df.columns)
x = cols.pop()

In [35]:
#converting the EJ to zero or one
def changer(string):
    ej_dict = {'A':0, 'B':1}
    return ej_dict[string]

df['EJ'] = df['EJ'].map(changer)

In [36]:
#filling the null values with median
for i in cols:
    if df[i].isnull().any():
        df[i].fillna(value=df[i].median(), inplace = True)
        

In [37]:
df.dtypes

AB       float64
AF       float64
AH       float64
AM       float64
AR       float64
AX       float64
AY       float64
AZ       float64
BC       float64
BD       float64
BN       float64
BP       float64
BQ       float64
BR       float64
BZ       float64
CB       float64
CC       float64
CD       float64
CF       float64
CH       float64
CL       float64
CR       float64
CS       float64
CU       float64
CW       float64
DA       float64
DE       float64
DF       float64
DH       float64
DI       float64
DL       float64
DN       float64
DU       float64
DV       float64
DY       float64
EB       float64
EE       float64
EG       float64
EH       float64
EJ         int64
EL       float64
EP       float64
EU       float64
FC       float64
FD       float64
FE       float64
FI       float64
FL       float64
FR       float64
FS       float64
GB       float64
GE       float64
GF       float64
GH       float64
GI       float64
GL       float64
Class      int64
dtype: object

In [38]:
y = df['Class']
df.drop(columns = ['Class'], inplace = True)



In [45]:
#use forward feature selection for upto 20 features (just to reduce data wala thing. for now that is. )

def get_accuracy_score(x,y):
    from xgboost import XGBClassifier as xgb
    from sklearn.model_selection import train_test_split as tts
    from sklearn.metrics import accuracy_score
    xgbclass = xgb()
    X_train, X_test, y_train, y_test = tts(x, y, test_size = 0.3, random_state = 0)
    xgbclass.fit(X_train,y_train)
    prediction = xgbclass.predict(X_test)
    acc = accuracy_score(prediction,y_test)
    return acc


def forward_feature_select(dataframe, y, number_of_features):
    from sklearn.preprocessing import StandardScaler as ssc
    scaler = ssc()

    max_feat = number_of_features
    counter = 1
    col_list = []
    temp_acc = 0
    temp_feat = ''
    for i in range(len(cols)):
        temp = cols[i]
        x = df[cols[i]]
        y = y
        x = np.asarray(x)
        y = np.asarray(y)
        x = np.reshape(x, (-1,1))
        x = scaler.fit_transform(x)
        
        #----------------------------#
        acc = get_accuracy_score(x,y)

        if acc>temp_acc:
            temp_acc = acc
            temp_feat = temp
        else:
            continue
    col_list.append(temp_feat)
    
    while counter<max_feat:
        tmanyacc = 0
        tempfeat = ''
        for i in range(len(cols)):
            if cols[i] in col_list:
                continue
            else:
                temp = cols[i]
                x = df[col_list+[cols[i]]]
                x = scaler.fit_transform(x)
                
                #---------------------#
                acc = get_accuracy_score(x,y)
                if acc>tmanyacc:
                    tmanyacc = acc
                    tempfeat = cols[i]
                else:
                    continue
        col_list.append(tempfeat)
        print(tmanyacc)
        counter+=1
    return col_list

In [87]:
feat_list = forward_feature_select(df,y,30)

0.8548387096774194
0.8870967741935484
0.9032258064516129
0.9247311827956989
0.9247311827956989
0.9193548387096774
0.9193548387096774
0.9193548387096774
0.946236559139785
0.946236559139785
0.946236559139785
0.946236559139785
0.9516129032258065
0.9516129032258065
0.9516129032258065
0.9516129032258065
0.9516129032258065
0.946236559139785
0.9516129032258065
0.956989247311828
0.956989247311828
0.956989247311828
0.956989247311828
0.967741935483871
0.9623655913978495
0.956989247311828
0.956989247311828
0.9623655913978495
0.9623655913978495


In [47]:
feat_list 

['CC',
 'DU',
 'BC',
 'EP',
 'FS',
 'EJ',
 'AH',
 'BQ',
 'EB',
 'DI',
 'EE',
 'CW ',
 'CR',
 'GI',
 'DN',
 'BZ',
 'CB',
 'DV',
 'BD ',
 'AR']

In [52]:
from sklearn.preprocessing import StandardScaler as ssc
scaler_main = ssc()

df_train = df[feat_list]

In [53]:
scaler_main = scaler_main.fit(df_train)

In [55]:
df_train = scaler_main.transform(df_train)

In [66]:
#testing the XGBoost Classifier

def get_accuracy_score_fin(x,y):
    from xgboost import XGBClassifier as xgb
    from sklearn.model_selection import train_test_split as tts
    from sklearn.metrics import accuracy_score
    xgbclass = xgb()
    X_train, X_test, y_train, y_test = tts(x, y, test_size = 0.3, random_state = 0)
    xgbclass = xgbclass.fit(X_train,y_train)
    prediction = xgbclass.predict(X_test)
    acc = accuracy_score(prediction,y_test)
    return acc,xgbclass

accuracy,model = get_accuracy_score_fin(df_train, y)


In [67]:
accuracy

0.9516129032258065

In [83]:
model.classes_

array([0, 1])

In [75]:
def test_preprocessing(testdataset):
    id_ = testdataset['Id']
    testdataset.drop(columns = ['Id'], axis = 1, inplace = True)
    testdataset['EJ'] = testdataset['EJ'].map(changer)
    for i in cols:
        if testdataset[i].isnull().any():
            testdataset[i].fillna(value=df[i].median(), inplace = True)
        
    testdataset = testdataset[feat_list]
    return testdataset,id_

def test(dataset):
    results =  model.predict_proba(dataset)
    return results

In [77]:
testo = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
testo,id_ = test_preprocessing(testo)
results = test(testo)
print(type(results))

<class 'numpy.ndarray'>


In [81]:
res = pd.DataFrame(results, columns=['class_0', 'class_1'])
fin = pd.concat([id_,res], axis = 1)

In [84]:
fin.to_csv('submission.csv', index = False)